In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
%cd /content/gdrive/My Drive/Contelligenz Data Science/movie_dataset

/content/gdrive/My Drive/Contelligenz Data Science/movie_dataset


In [5]:
import pandas as pd
import numpy as np
import pickle
import datetime as dt

In [6]:
contentMetadata = pd.read_csv('content_metadata_with_views_updated.csv')
contentMetadata.head()

,video_id,title_x,free,runtime,parentalRating,seriesId,episode_num,primary_category,category2,title_plink,publish_date,series_season,series_title_plink,video_category,views,seriesViews,uniqueViews,uniqueSeriesViews,userSeriesViews,userViews,video
0,15ee533f-00b0-4926-a0c0-5e09d609dec4,100 Days of Love,True,9163,U,NaN,NaN,Romance,Comedy,100 Days Of Love,27 April 2021,NaN,NaN,Movie,697.029357,0.000000,2068,NaN,NaN,1964,39
1,816ad052-68c9-4340-8aec-c357cfb76eff,Current Teega,True,7743,18+,NaN,NaN,Comedy,Romance,Current Teega,27 April 2021,NaN,NaN,Movie,153.955831,0.000000,625,NaN,NaN,615,225
2,41b17deb-e3c5-4453-ae15-c64f1f98a013,No.1 Yaari with Rana Episode 7,False,2793,NR,a15df463-f33c-4b5c-b85c-392f9f5093d7,e7,Talk show,NaN,No 1 Yaari With Rana,25 April 2021,Season 3,No 1 Yaari With Rana,Originals,1635.682062,7826.074127,2821,11028.0,9864.0,2687,111
3,ae94f7e7-8324-4962-8ada-bca0820d0603,Bhadram,True,7218,U,NaN,NaN,Mystery,NaN,Bhadram,23 April 2021,NaN,NaN,Movie,126.338321,0.000000,274,NaN,NaN,272,305
4,4c5ba7a3-4338-4b25-9a8b-1a6bf3c9fdb7,Selfie Raja,True,7186,13+,NaN,NaN,Comedy,NaN,Selfie Raja,23 April 2021,NaN,NaN,Movie,269.085722,0.000000,569,NaN,NaN,554,128


In [7]:
contentMetadata = contentMetadata.loc[contentMetadata['video_category'] == 'Originals']

In [8]:
contentMetadata = contentMetadata[['video_id', 'title_plink', 'seriesId', 'episode_num', 'runtime']]

In [9]:
contentMetadata['episode_count'] = contentMetadata.groupby('seriesId')['episode_num'].transform('nunique')

In [10]:
contentMetadata.head()

,video_id,title_plink,seriesId,episode_num,runtime,episode_count
2,41b17deb-e3c5-4453-ae15-c64f1f98a013,No 1 Yaari With Rana,a15df463-f33c-4b5c-b85c-392f9f5093d7,e7,2793,7
13,5caeaab3-35bb-4fb7-b746-725fc9aff10c,No 1 Yaari With Rana,a15df463-f33c-4b5c-b85c-392f9f5093d7,e6,3212,7
15,fd1f8531-54f7-42a8-b6eb-ec62daec91a7,No 1 Yaari With Rana,a15df463-f33c-4b5c-b85c-392f9f5093d7,e5,2749,7
16,c0a46111-3be3-4cce-851b-a17c656ca680,11Th Hour,1bf00638-81f4-456b-8d51-e36387a7f7a3,e6,1152,8
17,519e72e8-ac1c-4bbe-9890-5a5a85ac2fa6,11Th Hour,1bf00638-81f4-456b-8d51-e36387a7f7a3,e8,2083,8


In [11]:
contentMetadata[contentMetadata['title_plink'] == 'Locked']

,video_id,title_plink,seriesId,episode_num,runtime,episode_count
371,8c3fa90d-321a-49b6-b9b0-186a6492b571,Locked,19e15423-40f6-419a-b97d-2754fd04ff4d,e1,1887,7
372,6e119257-4599-45f5-878f-2b6ab98ac480,Locked,19e15423-40f6-419a-b97d-2754fd04ff4d,e7,1561,7
373,65bdf1f6-3908-4f12-a418-fb412dd03d75,Locked,19e15423-40f6-419a-b97d-2754fd04ff4d,e6,1582,7
374,71930ab0-d90c-4056-b37c-d7778ce35aee,Locked,19e15423-40f6-419a-b97d-2754fd04ff4d,e5,1448,7
375,e95bcf76-9880-4dba-8554-058783e9d489,Locked,19e15423-40f6-419a-b97d-2754fd04ff4d,e4,1705,7
376,7ae38474-f70e-48b4-b70d-26d0435486be,Locked,19e15423-40f6-419a-b97d-2754fd04ff4d,e3,1847,7
378,9158af56-c8fa-4077-9964-dbf40fd29fa7,Locked,19e15423-40f6-419a-b97d-2754fd04ff4d,e2,1468,7


In [12]:
contentMetadata.loc[contentMetadata['title_plink'].str.contains('Kothaporadu', case = False), 'title_plink'] = 'Kotha Poradu'

In [13]:
%cd /content/gdrive/My Drive/Contelligenz Data Science/Web-series analysis

/content/gdrive/My Drive/Contelligenz Data Science/Web-series analysis


In [14]:
userEvents = pickle.load(open('user_events_raw_all.pkl','rb'))
userEvents.head()

,streamid,contentid,userid,subscription_plan_id,watchtime,duration,browser,devicename,country,cityname,subdivisionname
0,04d0a064-b96b-3f49-a8f3-d1f692a9fbb7,6e119257-4599-45f5-878f-2b6ab98ac480,edce2372-1a65-4333-984c-5d5d831e622d,None,2021-05-25 12:33:06,570,,Samsung SM-E625F,IN,Tanuku,Andhra Pradesh
1,edc77967-30a3-3155-87da-141b6441b0d5,ab6c94cd-8000-419e-995c-f97f07ac6ce5,edce2372-1a65-4333-984c-5d5d831e622d,None,2021-05-24 10:37:38,360,,Samsung SM-E625F,IN,Tanuku,Andhra Pradesh
2,9ef047ac-c370-3406-9d8f-1282a82f9157,948927a5-7e54-49f9-9abb-79f1e749acfc,edce2372-1a65-4333-984c-5d5d831e622d,None,2021-05-24 00:09:41,360,,Samsung SM-E625F,IN,Tanuku,Andhra Pradesh
3,652eaa8e-1b83-305c-a2b7-9dc2c81cd348,948927a5-7e54-49f9-9abb-79f1e749acfc,edce2372-1a65-4333-984c-5d5d831e622d,None,2021-05-24 00:00:11,420,,Samsung SM-E625F,IN,Hyderabad,Telangana
4,39aefc2e-0fff-32bb-872d-b33fa0d9a6c2,9b79eb7c-6ca9-4f0a-b1cb-c4afa4ffdffb,edce2372-1a65-4333-984c-5d5d831e622d,None,2021-05-23 23:36:21,240,,Samsung SM-E625F,IN,Hyderabad,Telangana


In [15]:
userEvents = userEvents.rename(columns = {"contentid":"video_id"})

In [16]:
userEvents.drop(['subscription_plan_id', 'browser', 'devicename', 'country', 'cityname', 'subdivisionname'], axis = 1, inplace = True)

In [17]:
df = pd.merge(contentMetadata, userEvents, on = 'video_id', how = 'left')

In [18]:
df.head()

,video_id,title_plink,seriesId,episode_num,runtime,episode_count,streamid,userid,watchtime,duration
0,41b17deb-e3c5-4453-ae15-c64f1f98a013,No 1 Yaari With Rana,a15df463-f33c-4b5c-b85c-392f9f5093d7,e7,2793,7,db873cff-b9ed-39e5-b27f-3ce2bf9a5a3f,edd12f8d-5820-4381-a371-447f56465a95,2021-05-12 14:29:20,2640
1,41b17deb-e3c5-4453-ae15-c64f1f98a013,No 1 Yaari With Rana,a15df463-f33c-4b5c-b85c-392f9f5093d7,e7,2793,7,8cc2327d-8f6e-3ce4-a77d-b71beff0186a,eddace06-c471-4e7d-9222-1b72f0ece800,2021-05-06 12:02:05,0
2,41b17deb-e3c5-4453-ae15-c64f1f98a013,No 1 Yaari With Rana,a15df463-f33c-4b5c-b85c-392f9f5093d7,e7,2793,7,8c0b45789810ca4413fa218ebdcb9dd0cdc16c3c,ee1063d4-9814-4fed-80bc-ae155dd73a4d,2021-05-22 11:48:09,60
3,41b17deb-e3c5-4453-ae15-c64f1f98a013,No 1 Yaari With Rana,a15df463-f33c-4b5c-b85c-392f9f5093d7,e7,2793,7,86dbdfdf-ff35-3472-a56e-50092f2a38e6,ee118390-aad4-45d2-b435-6f7fd85301b7,2021-05-18 09:20:57,2520
4,41b17deb-e3c5-4453-ae15-c64f1f98a013,No 1 Yaari With Rana,a15df463-f33c-4b5c-b85c-392f9f5093d7,e7,2793,7,77328de1-ea96-d4a9-86d1-c2db77141c5f,ee164b9f-b581-4d67-a02e-52eb5ec798a4,2021-05-04 10:38:00,2220


In [19]:
df['watchtime'] = df['watchtime'].dt.date

In [20]:
df_grouped = df.groupby(['userid', 'seriesId', 'title_plink']).agg({'video_id':'first', 'episode_num':'first', 'runtime':'first', 'watchtime':'first', 'duration':'first'})

In [21]:
df_grouped.head()

video_id  ... duration
userid                               seriesId                             title_plink                                                 ...         
ebbbe22e-0936-45dc-91eb-d67d0e9f3a90 19e15423-40f6-419a-b97d-2754fd04ff4d Locked                e95bcf76-9880-4dba-8554-058783e9d489  ...      210
ebbc6236-c635-4244-a5b7-0ccfcda54056 8a1a9dde-6b45-4596-ab3a-a244c6da649f Gs 2                  1544fcbc-cd48-4f6c-9515-a7d98156fcf1  ...     1440
                                     a15df463-f33c-4b5c-b85c-392f9f5093d7 No 1 Yaari With Rana  25f5436c-bc50-482e-8f58-124a22eaca75  ...      510
                                     bad87983-d2a3-4c3e-9af2-4341af94042d Mastis                b7a6f226-64b0-4812-a125-6378aca50a88  ...       90
ebbec721-cb12-44e4-924d-a241ef83d07c 1bf00638-81f4-456b-8d51-e36387a7f7a3 11Th Hour             c0a46111-3be3-4cce-851b-a17c656ca680  ...      930

[5 rows x 5 columns]

In [22]:
## Average Completion rate: Per user - per web series
df_1 = df.groupby(['seriesId', 'title_plink', 'episode_num']).agg({'runtime':'mean', 'duration':'mean', 'episode_count':'first'})

In [23]:
df_1['completionRatio'] = df_1['duration'] / df_1['runtime']

In [24]:
### Granular - For every episode
#df_1[15:25]
sum([1280, 1120, 1123, 1060, 1253, 1260, 1457, 1043, 1239, 1374])

12209

In [25]:
### Based on web-series
### Dropping the rows where duration is equal to 0
### There are 60500 such records where duration is 0!!

df2 = df[df['duration'] > 0]
df_2 = df2.groupby(['seriesId', 'title_plink']).agg({'runtime':lambda x: x.unique().mean(), 'duration':'mean'})   #### GIves average runtime and duration of each episode.
df_2['completionRatio'] = df_2['duration'] / df_2['runtime']
df_2.sort_values('completionRatio', ascending = False) 

#df_2 = df2.groupby(['seriesId', 'title_plink']).agg({'runtime':lambda x: x.unique().sum(), 'duration':'mean'})
#df_2

,,runtime,duration,completionRatio
seriesId,title_plink,,,
8a1a9dde-6b45-4596-ab3a-a244c6da649f,Gs 2,1436.500000,996.366984,0.693607
eaad44f8-1ef1-4ad4-abba-12470aaa405f,Shit Happens,1160.000000,787.931034,0.679251
93fb3670-e354-4853-a193-7e32e55c20e6,Addham,1134.666667,734.143845,0.647013
1bf00638-81f4-456b-8d51-e36387a7f7a3,11Th Hour,1449.500000,911.790181,0.629038
19e15423-40f6-419a-b97d-2754fd04ff4d,Locked,1642.571429,1017.154927,0.619245
bad87983-d2a3-4c3e-9af2-4341af94042d,Mastis,1457.875000,850.502372,0.583385
368c4a31-6dbf-4318-ba79-b39af9af27fe,Kotha Poradu,1220.900000,693.267758,0.567833
3a2096da-5401-4aab-8f71-5560d0b5d0ba,Commitmental,1657.000000,916.242509,0.552953
9d19c381-e299-4a48-908c-78506aa8d74e,Honeymoon,1356.000000,746.552627,0.550555


In [26]:
df_2 = df_2.reset_index()
df_2

,seriesId,title_plink,runtime,duration,completionRatio
0,19e15423-40f6-419a-b97d-2754fd04ff4d,Locked,1642.571429,1017.154927,0.619245
1,1bf00638-81f4-456b-8d51-e36387a7f7a3,11Th Hour,1449.500000,911.790181,0.629038
2,368c4a31-6dbf-4318-ba79-b39af9af27fe,Kotha Poradu,1220.900000,693.267758,0.567833
3,3a2096da-5401-4aab-8f71-5560d0b5d0ba,Commitmental,1657.000000,916.242509,0.552953
4,69644ab8-af79-46ef-84b8-82895e6dd9a1,All Is Well,1002.000000,479.142857,0.478186
5,8a1a9dde-6b45-4596-ab3a-a244c6da649f,Gs 2,1436.500000,996.366984,0.693607
6,921e1d91-57f5-412a-bc58-57f415f45e78,Sin,1164.000000,529.576154,0.454962
7,93fb3670-e354-4853-a193-7e32e55c20e6,Addham,1134.666667,734.143845,0.647013
8,9ade7795-a8f4-4959-ba34-ac0c09b3fa24,Sam Jam,3252.666667,1287.705038,0.395892
9,9d19c381-e299-4a48-908c-78506aa8d74e,Honeymoon,1356.000000,746.552627,0.550555


In [27]:
import plotly.graph_objects as go

In [28]:
fig = go.Figure(data = [
                        go.Bar(name='Avg. Runtime', x = df_2['title_plink'], y = df_2['runtime']),
                        go.Bar(name='Avg. Duration', x = df_2['title_plink'], y = df_2['duration'], hovertext = df_2['completionRatio']),
                        
])
fig.update_layout(barmode = 'group')
fig.show()

In [29]:
df3 = df[df['duration'] > 0]
df3.drop(['streamid'], axis = 1, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [30]:
df3.head()

,video_id,title_plink,seriesId,episode_num,runtime,episode_count,userid,watchtime,duration
0,41b17deb-e3c5-4453-ae15-c64f1f98a013,No 1 Yaari With Rana,a15df463-f33c-4b5c-b85c-392f9f5093d7,e7,2793,7,edd12f8d-5820-4381-a371-447f56465a95,2021-05-12,2640
2,41b17deb-e3c5-4453-ae15-c64f1f98a013,No 1 Yaari With Rana,a15df463-f33c-4b5c-b85c-392f9f5093d7,e7,2793,7,ee1063d4-9814-4fed-80bc-ae155dd73a4d,2021-05-22,60
3,41b17deb-e3c5-4453-ae15-c64f1f98a013,No 1 Yaari With Rana,a15df463-f33c-4b5c-b85c-392f9f5093d7,e7,2793,7,ee118390-aad4-45d2-b435-6f7fd85301b7,2021-05-18,2520
4,41b17deb-e3c5-4453-ae15-c64f1f98a013,No 1 Yaari With Rana,a15df463-f33c-4b5c-b85c-392f9f5093d7,e7,2793,7,ee164b9f-b581-4d67-a02e-52eb5ec798a4,2021-05-04,2220
5,41b17deb-e3c5-4453-ae15-c64f1f98a013,No 1 Yaari With Rana,a15df463-f33c-4b5c-b85c-392f9f5093d7,e7,2793,7,ee16c54d-9c57-47e5-8c56-9a6853836556,2021-05-23,450


In [31]:
df_3 = df3.groupby(['userid', 'seriesId']).agg({'episode_num':'first', 'runtime':'first', 'duration':'first', 'watchtime':'first', 'episode_count':'first'})

In [32]:
df_3.head()

episode_num  ...  episode_count
userid                               seriesId                                          ...               
ebbbe22e-0936-45dc-91eb-d67d0e9f3a90 19e15423-40f6-419a-b97d-2754fd04ff4d          e4  ...              7
ebbc6236-c635-4244-a5b7-0ccfcda54056 8a1a9dde-6b45-4596-ab3a-a244c6da649f          e3  ...              8
                                     a15df463-f33c-4b5c-b85c-392f9f5093d7          e1  ...              7
                                     bad87983-d2a3-4c3e-9af2-4341af94042d          e5  ...              8
ebbec721-cb12-44e4-924d-a241ef83d07c 1bf00638-81f4-456b-8d51-e36387a7f7a3          e6  ...              8

[5 rows x 5 columns]

In [33]:
### Binged-watched
#df4 = df3.groupby(['userid', 'seriesId', 'watchtime']).agg({'episode_num': lambda x: list(set(x)), 'runtime':[lambda x: list(set(x)), 'sum'], 'duration':[lambda x: list(set(x)), 'sum'], 'episode_count':'first'})

df4 = df3.groupby(['userid', 'seriesId', 'watchtime']).agg(ep_lst = ('episode_num', lambda x: list(set(x))), runtimes_lst = ('runtime', lambda x: list(set(x))),
                                                           runtime_sum = ('runtime', 'sum'), duration_lst = ('duration', lambda x: list(set(x))), 
                                                           duration_sum = ('duration', 'sum'), episdoe_count = ('episode_count','first'))


In [34]:
df4.head()

ep_lst  ... episdoe_count
userid                               seriesId                             watchtime                                     ...              
ebbbe22e-0936-45dc-91eb-d67d0e9f3a90 19e15423-40f6-419a-b97d-2754fd04ff4d 2021-05-23                              [e4]  ...             7
ebbc6236-c635-4244-a5b7-0ccfcda54056 8a1a9dde-6b45-4596-ab3a-a244c6da649f 2021-05-21                      [e2, e3, e1]  ...             8
                                     a15df463-f33c-4b5c-b85c-392f9f5093d7 2021-05-10                              [e1]  ...             7
                                     bad87983-d2a3-4c3e-9af2-4341af94042d 2021-05-11  [e8, e2, e6, e1, e5, e7, e3, e4]  ...             8
ebbec721-cb12-44e4-924d-a241ef83d07c 1bf00638-81f4-456b-8d51-e36387a7f7a3 2021-05-02          [e8, e6, e5, e7, e3, e4]  ...             8

[5 rows x 6 columns]

In [35]:
df4['watched_episode_count'] = df4['ep_lst'].str.len()

In [36]:
df4.head()

ep_lst  ... watched_episode_count
userid                               seriesId                             watchtime                                     ...                      
ebbbe22e-0936-45dc-91eb-d67d0e9f3a90 19e15423-40f6-419a-b97d-2754fd04ff4d 2021-05-23                              [e4]  ...                     1
ebbc6236-c635-4244-a5b7-0ccfcda54056 8a1a9dde-6b45-4596-ab3a-a244c6da649f 2021-05-21                      [e2, e3, e1]  ...                     3
                                     a15df463-f33c-4b5c-b85c-392f9f5093d7 2021-05-10                              [e1]  ...                     1
                                     bad87983-d2a3-4c3e-9af2-4341af94042d 2021-05-11  [e8, e2, e6, e1, e5, e7, e3, e4]  ...                     8
ebbec721-cb12-44e4-924d-a241ef83d07c 1bf00638-81f4-456b-8d51-e36387a7f7a3 2021-05-02          [e8, e6, e5, e7, e3, e4]  ...                     6

[5 rows x 7 columns]

In [37]:
### Mark records as 1 wherre users have wached more than 2 episodes in a day.
df4['binged'] = np.where(df4['watched_episode_count'] >= 2, 1, 0)

In [38]:
df4.head()

ep_lst  ... binged
userid                               seriesId                             watchtime                                     ...       
ebbbe22e-0936-45dc-91eb-d67d0e9f3a90 19e15423-40f6-419a-b97d-2754fd04ff4d 2021-05-23                              [e4]  ...      0
ebbc6236-c635-4244-a5b7-0ccfcda54056 8a1a9dde-6b45-4596-ab3a-a244c6da649f 2021-05-21                      [e2, e3, e1]  ...      1
                                     a15df463-f33c-4b5c-b85c-392f9f5093d7 2021-05-10                              [e1]  ...      0
                                     bad87983-d2a3-4c3e-9af2-4341af94042d 2021-05-11  [e8, e2, e6, e1, e5, e7, e3, e4]  ...      1
ebbec721-cb12-44e4-924d-a241ef83d07c 1bf00638-81f4-456b-8d51-e36387a7f7a3 2021-05-02          [e8, e6, e5, e7, e3, e4]  ...      1

[5 rows x 8 columns]

## Time taken to watch whole series

In [39]:
### Get Sum of runtime of all web-series
df_5 = df.groupby(['seriesId']).agg({'runtime':lambda x: x.unique().sum()})

In [40]:
df_5 = df_5.rename(columns = {'runtime':'series_runtime'})

In [41]:
df_5 = df_5.reset_index()
df_5

,seriesId,series_runtime
0,19e15423-40f6-419a-b97d-2754fd04ff4d,11498
1,1bf00638-81f4-456b-8d51-e36387a7f7a3,11596
2,368c4a31-6dbf-4318-ba79-b39af9af27fe,12209
3,3a2096da-5401-4aab-8f71-5560d0b5d0ba,8285
4,69644ab8-af79-46ef-84b8-82895e6dd9a1,8016
5,8a1a9dde-6b45-4596-ab3a-a244c6da649f,11492
6,921e1d91-57f5-412a-bc58-57f415f45e78,8148
7,93fb3670-e354-4853-a193-7e32e55c20e6,3404
8,9ade7795-a8f4-4959-ba34-ac0c09b3fa24,29274
9,9d19c381-e299-4a48-908c-78506aa8d74e,8136


In [42]:
df6 = pd.merge(df_5, df3, on = 'seriesId', how = 'left')

In [43]:
df6.head()

,seriesId,series_runtime,video_id,title_plink,episode_num,runtime,episode_count,userid,watchtime,duration
0,19e15423-40f6-419a-b97d-2754fd04ff4d,11498,8c3fa90d-321a-49b6-b9b0-186a6492b571,Locked,e1,1887,7,edce2372-1a65-4333-984c-5d5d831e622d,2021-05-18,900
1,19e15423-40f6-419a-b97d-2754fd04ff4d,11498,8c3fa90d-321a-49b6-b9b0-186a6492b571,Locked,e1,1887,7,edefe96f-fd12-49d9-9f0c-94e5f9269f01,2021-05-02,690
2,19e15423-40f6-419a-b97d-2754fd04ff4d,11498,8c3fa90d-321a-49b6-b9b0-186a6492b571,Locked,e1,1887,7,edefe96f-fd12-49d9-9f0c-94e5f9269f01,2021-05-02,1410
3,19e15423-40f6-419a-b97d-2754fd04ff4d,11498,8c3fa90d-321a-49b6-b9b0-186a6492b571,Locked,e1,1887,7,ee2bfb64-d747-4506-bd61-5d71fdea3955,2021-05-04,1890
4,19e15423-40f6-419a-b97d-2754fd04ff4d,11498,8c3fa90d-321a-49b6-b9b0-186a6492b571,Locked,e1,1887,7,ee46788e-a6c7-4205-a40d-9f3313c9fb1d,2021-05-25,1440


In [44]:
df5 = df6.groupby(['userid', 'seriesId']).agg(series_runtime = ('series_runtime', 'first'), ep_lst = ('episode_num', lambda x: list(set(x))), 
                                              runtimes_lst = ('runtime', lambda x: list(set(x))), 
                                              runtime_sum = ('runtime', lambda x: x.unique().sum()), duration_lst = ('duration', lambda x: list(x)), 
                                              duration_sum = ('duration', lambda y: y.unique().sum()), series_episodes = ('episode_count','first'),
                                              watchtime_lst = ('watchtime', lambda x: list(set(x))))

In [45]:
df5['watched_episode_count'] = df5['ep_lst'].str.len()
df5['days_count'] = df5['watchtime_lst'].str.len()
df5['duration_lst_count'] = df5['duration_lst'].str.len()

In [46]:
df5.head()

series_runtime  ... duration_lst_count
userid                               seriesId                                              ...                   
ebbbe22e-0936-45dc-91eb-d67d0e9f3a90 19e15423-40f6-419a-b97d-2754fd04ff4d           11498  ...                  2
ebbc6236-c635-4244-a5b7-0ccfcda54056 8a1a9dde-6b45-4596-ab3a-a244c6da649f           11492  ...                  3
                                     a15df463-f33c-4b5c-b85c-392f9f5093d7           20310  ...                  1
                                     bad87983-d2a3-4c3e-9af2-4341af94042d           11663  ...                 14
ebbec721-cb12-44e4-924d-a241ef83d07c 1bf00638-81f4-456b-8d51-e36387a7f7a3           11596  ...                  6

[5 rows x 11 columns]

In [47]:
temp = df5.loc[df5['watched_episode_count'] >= df5['series_episodes']]

In [48]:
temp = temp.copy()
temp['watch_ratio'] = temp['duration_sum'] / temp['runtime_sum']

In [49]:
temp.head()

series_runtime  ... watch_ratio
userid                               seriesId                                              ...            
ebbc6236-c635-4244-a5b7-0ccfcda54056 bad87983-d2a3-4c3e-9af2-4341af94042d           11663  ...    0.347252
ebc0d993-af9c-4932-878e-ed8c15e860b2 921e1d91-57f5-412a-bc58-57f415f45e78            8148  ...    0.143594
ebc422df-8a99-4231-b122-765e5b4ccf2c 368c4a31-6dbf-4318-ba79-b39af9af27fe           12209  ...    1.287575
ebc7a96e-910c-486f-b753-81c377e6c7c9 19e15423-40f6-419a-b97d-2754fd04ff4d           11498  ...    1.048878
                                     bad87983-d2a3-4c3e-9af2-4341af94042d           11663  ...    0.923433

[5 rows x 12 columns]

In [50]:
len(temp.watch_ratio)

14102

In [51]:
len(temp[temp['watch_ratio'] > 0.8])

6282

In [52]:
### A web-series is considered watched completely, if viewer has watched all episodes and the duration spent watching is greater than 0.8 * total runtime of the we-seres.

temp['watchedCompletely'] = np.where((temp['watched_episode_count'] >= temp['series_episodes']) & (temp['watch_ratio'] > 0.8), 1, 0)

In [53]:
temp['avg_session_time'] = temp['duration_sum'] / temp['duration_lst_count']

In [54]:
temp.head()

series_runtime  ... avg_session_time
userid                               seriesId                                              ...                 
ebbc6236-c635-4244-a5b7-0ccfcda54056 bad87983-d2a3-4c3e-9af2-4341af94042d           11663  ...       289.285714
ebc0d993-af9c-4932-878e-ed8c15e860b2 921e1d91-57f5-412a-bc58-57f415f45e78            8148  ...       146.250000
ebc422df-8a99-4231-b122-765e5b4ccf2c 368c4a31-6dbf-4318-ba79-b39af9af27fe           12209  ...       507.096774
ebc7a96e-910c-486f-b753-81c377e6c7c9 19e15423-40f6-419a-b97d-2754fd04ff4d           11498  ...      1096.363636
                                     bad87983-d2a3-4c3e-9af2-4341af94042d           11663  ...      1077.000000

[5 rows x 14 columns]